In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import numpy as np
import scipy.sparse as sp

from solardatatools import DataHandler
from solardatatools.dataio import load_redshift_data
from solardatatools.algorithms import Dilation
from spcqe.quantiles import SmoothPeriodicQuantiles
from spcqe.functions import make_basis_matrix, make_regularization_matrix

In [3]:
directory = 'fixtures'

## Basis matrix

3 periods and a trend term

In [4]:
basis_a1 = make_basis_matrix(num_harmonics=10, length=500, periods=[11, 17, 23], trend=True)
np.save(os.path.join(directory, 'basis_a1.npy'), basis_a1)

2 periods and no trend term

In [5]:
basis_a2 = make_basis_matrix(num_harmonics=10, length=400, periods=[11, 17], trend=False)
np.save(os.path.join(directory, 'basis_a2.npy'), basis_a2)

multiple harmonics and standing wave

In [6]:
basis_a3 = make_basis_matrix(num_harmonics=[3, 7], length=370, periods=[11, 3], standing_wave=[True, False], trend=False)
np.save(os.path.join(directory, 'basis_a3.npy'), basis_a3)

multiple harmonics and standing wave and max cross terms

In [7]:
basis_a4 = make_basis_matrix(num_harmonics=[4, 3], length=420, periods=[6, 8], standing_wave=True, trend=True, max_cross_k=4)
np.save(os.path.join(directory, 'basis_a4.npy'), basis_a4)

## Regularization matrix

3 periods and a trend term

In [8]:
reg_a1 = make_regularization_matrix(num_harmonics=10, weight=10, periods=[11, 17, 23], trend=True)
sp.save_npz(os.path.join(directory, 'reg_a1.npz'), reg_a1)

2 periods and no trend term

In [9]:
reg_a2 = make_regularization_matrix(num_harmonics=10, weight=1, periods=[11, 17], trend=False)
sp.save_npz(os.path.join(directory, 'reg_a2.npz'), reg_a2)

multiple harmonics and standing wave

In [10]:
reg_a3 = make_regularization_matrix(num_harmonics=[3, 7], weight=0, periods=[11, 3], standing_wave=[True, False], trend=False)
sp.save_npz(os.path.join(directory, 'reg_a3.npz'), reg_a3)

multiple harmonics and standing wave and max cross terms

In [11]:
reg_a4 = make_regularization_matrix(num_harmonics=[4, 3], weight=3, periods=[6, 8], standing_wave=True, trend=True, max_cross_k=4)
sp.save_npz(os.path.join(directory, 'reg_a4.npz'), reg_a4)

## Tranforming pv data

Import

In [12]:
query = {
'siteid': "TADBC1078041",
'api_key': os.environ.get('REDSHIFT_API_KEY'),
'sensor': 0
}
df = load_redshift_data(**query)
dh = DataHandler(df, convert_to_ts=True)
dh.run_pipeline()

/opt/miniconda3/envs/spcqe-dev/lib/python3.11/site-packages/solardatatools/time_axis_manipulation.py:79: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  meas_name = str(df_view[name_key][0])



            *********************************************
            * Solar Data Tools Data Onboarding Pipeline *
            *********************************************

            This pipeline runs a series of preprocessing, cleaning, and quality
            control tasks on stand-alone PV power or irradiance time series data.
            After the pipeline is run, the data may be plotted, filtered, or
            further analyzed.

            Authors: Bennet Meyers and Sara Miskovich, SLAC

            (Tip: if you have a mosek [https://www.mosek.com/] license and have it
            installed on your system, try setting solver='MOSEK' for a speedup)

            This material is based upon work supported by the U.S. Department
            of Energy's Office of Energy Efficiency and Renewable Energy (EERE)
            under the Solar Energy Technologies Office Award Number 38529.

            


task list:   0%|                                          | 0/7 [00:00<?, ?it/s]/opt/miniconda3/envs/spcqe-dev/lib/python3.11/site-packages/solardatatools/time_axis_manipulation.py:146: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if avg_day[0] >= thresh:
/opt/miniconda3/envs/spcqe-dev/lib/python3.11/site-packages/solardatatools/time_axis_manipulation.py:152: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if avg_day[-1] >= thresh:
/opt/miniconda3/envs/spcqe-dev/lib/python3.11/site-packages/solardatatools/matrix_embedding.py:41: FutureWarning: Second.delta is deprecated and will be removed in a future version. Use pd.



total time: 9.63 seconds
--------------------------------
Breakdown
--------------------------------
Preprocessing              3.15s
Cleaning                   0.18s
Filtering/Summarizing      6.30s
    Data quality           0.10s
    Clear day detect       0.21s
    Clipping detect        1.50s
    Capacity change detect 4.50s



Dilation

In [13]:
nvals_dil = 101
dil = Dilation(dh)
sig = dil.signal_dil
ndays = 50
sig = sig[:1 + nvals_dil * ndays]

In [14]:
np.save(os.path.join(directory, 'pvsig.npy'), sig)

Transform

In [15]:
my_quantiles = [0.02, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.98]

In [16]:
spq1 = SmoothPeriodicQuantiles(
        num_harmonics=[10,3],
        periods=[nvals_dil, 365.24225*nvals_dil],
        standing_wave=[True, False],
        trend=False,
        quantiles=my_quantiles,
        weight=10,
        problem='sequential',
        solver='qss',
        verbose=False,
)
spq1.fit(sig)

YES


100%|███████████████████████████████████████████| 11/11 [00:07<00:00,  1.51it/s]


In [17]:
transformed_data1 = spq1.transform(sig)
np.save(os.path.join(directory, 'transformed_data1.npy'), transformed_data1)

In [20]:
spq2 = SmoothPeriodicQuantiles(
        num_harmonics=[10,3],
        periods=[nvals_dil, 365.24225*nvals_dil],
        standing_wave=[True, False],
        trend=False,
        quantiles=my_quantiles,
        weight=10,
        problem='sequential',
        solver='mosek',
        verbose=True
)
spq2.fit(sig)

YES


  0%|                                                    | 0/11 [00:00<?, ?it/s]

                                     CVXPY                                     
                                     v1.4.3                                    
(CVXPY) May 19 09:23:01 PM: Your problem has 10179 variables, 1 constraints, and 0 parameters.
(CVXPY) May 19 09:23:01 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 19 09:23:01 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 19 09:23:01 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) May 19 09:23:01 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 19 09:23:01 PM: Compiling problem (target solver=MOSEK).


  0%|                                                    | 0/11 [01:00<?, ?it/s]

ValueError: Invalid keyword-argument(s) dict_keys(['rho_update', 'rho', 'max_iter', 'eps_abs', 'eps_rel']) passed to MOSEK solver.

In [19]:
transformed_data2 = spq2.transform(sig)
np.save(os.path.join(directory, 'transformed_data2.npy'), transformed_data2)

In [18]:
spq3 = SmoothPeriodicQuantiles(
        num_harmonics=[10,3],
        periods=[nvals_dil, 365.24225*nvals_dil],
        standing_wave=[True, False],
        trend=False,
        quantiles=my_quantiles,
        weight=10,
        problem='full',
        solver='mosek'
)
spq3.fit(sig)

In [21]:
transformed_data3 = spq3.transform(sig)
np.save(os.path.join(directory, 'transformed_data3.npy'), transformed_data3)

In [23]:
spq4 = SmoothPeriodicQuantiles(
        num_harmonics=[10,3],
        periods=[nvals_dil, 365.24225*nvals_dil],
        standing_wave=[True, False],
        trend=False,
        quantiles=my_quantiles,
        weight=10,
        problem='full',
        solver='clarabel'
)
spq4.fit(sig)

In [24]:
transformed_data4 = spq4.transform(sig)
np.save(os.path.join(directory, 'transformed_data4.npy'), transformed_data4)